In [ ]:
from Utils.Utils import *
from Utils.Blacksmith import * 

from Utils.HyMNet import HyMNet
from sklearn.svm import SVC

In [ ]:
# Seed
set_seed(0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
PATH = "/home/baharoon/HTN/data/"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = "/home/baharoon/HTN/Models"

os.makedirs(MODELS_PATH, exist_ok=True)

In [ ]:
BATCH_SIZE = 16
epochs = 50

image_size = 586
crop_size = 512

train_transform = T.Compose([
    T.Resize((image_size, image_size)),
    T.CenterCrop(crop_size),
    T.ToTensor(),
    T.RandomHorizontalFlip(0.5),
    T.RandomRotation(degrees=(0, 360)),
    T.GaussianBlur(3),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = T.Compose([
    T.Resize((image_size, image_size)),
    T.CenterCrop(crop_size),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = HypertensionDataset(CSV_PATH, split="train", train_transform=train_transform)
# val_dataset = HypertensionDataset(CSV_PATH, split="val", test_transform=test_transform)
val_dataset = HypertensionDataset(CSV_PATH, split="val", test_transform=train_transform)
test_dataset = HypertensionDataset(CSV_PATH, split="test", test_transform=test_transform)

train_dataset = torch.utils.data.ConcatDataset([train_dataset, val_dataset])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

epoch_length = math.ceil(len(train_dataset) / BATCH_SIZE)

In [ ]:
FM_PATH = MODELS_PATH + r"/FundusModel.pth"
DM_PATH = MODELS_PATH + r"/DemographicFCNN.pth"

criterion = nn.BCEWithLogitsLoss()

# FeatureFusion (Joint Fusion)

In [ ]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=8, out_features=32),
)

image_model = get_retfound("/home/baharoon/HTN/RETFound_cfp_weights.pth", image_size=512,
                          classes=8).requires_grad_(True)

fusion_model = nn.Sequential(
    nn.Linear(in_features=40, out_features=128),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

In [ ]:
model = HyMNet(image_model=image_model, tabular_model=tabular_model, fusion_model=fusion_model)

all_params_image = dict(model.image_model.named_parameters())
head_params_image = dict(model.image_model.head.named_parameters())

del all_params_image['head.weight']
del all_params_image['head.bias']

# Parallelize model to multiple GPUs
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    
model.to(device)

optimizer = torch.optim.AdamW([
    {'params': all_params_image.values(), 'lr': 1e-6},
    {'params': head_params_image.values(), 'lr': 0.005},
    {'params': tabular_model.parameters(), 'lr': 0.005},
    {'params': fusion_model.parameters(), 'lr': 0.005},
])

In [ ]:
epochs = 50
epoch_length = math.ceil(len(train_dataset) / BATCH_SIZE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epoch_length * epochs, eta_min=0)
metrics, best_model = train_val(epochs=epochs, model=model, criterion=criterion, optimizer=optimizer, train_loader=train_loader,
                        val_loader=test_loader, scheduler=scheduler, device=device, save_model=True)

In [ ]:
torch.save(best_model, MODELS_PATH+ "/JointFusion_finetune.pth")

# PredictionFusion (Late Fusion)

In [ ]:
tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=8, out_features=16),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=16, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=8, out_features=1),
)

image_model = get_retfound("/home/baharoon/HTN/RETFound_cfp_weights.pth", image_size=512,
                          classes=1).requires_grad_(True)

fusion_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=8, out_features=32),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

In [ ]:
model = HyMNet(image_model=image_model, tabular_model=tabular_model, fusion_model=fusion_model)

all_params_image = dict(model.image_model.named_parameters())
head_params_image = dict(model.image_model.head.named_parameters())

del all_params_image['head.weight']
del all_params_image['head.bias']

# Parallelize model to multiple GPUs
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    
model.to(device)

optimizer = torch.optim.AdamW([
    {'params': all_params_image.values(), 'lr': 1e-6},
    {'params': head_params_image.values(), 'lr': 0.005},
    {'params': tabular_model.parameters(), 'lr': 0.005},
    {'params': fusion_model.parameters(), 'lr': 0.005},
])

In [ ]:
epochs = 50
epoch_length = math.ceil(len(train_dataset) / BATCH_SIZE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epoch_length * epochs, eta_min=0)
metrics, best_model = train_val(epochs=epochs, model=model, criterion=criterion, optimizer=optimizer, train_loader=train_loader,
                        val_loader=test_loader, scheduler=scheduler, device=device, save_model=True)

In [ ]:
torch.save(best_model, MODELS_PATH+ "/PredFusion.pth")

## LateFusion

In [ ]:
image_model = get_retfound("/home/baharoon/HTN/RETFound_cfp_weights.pth", image_size=512,
                          classes=1).requires_grad_(True)
model = HyMNet(image_model=image_model).to(device)
state_dict = torch.load(FM_PATH)
state_dict = {key.replace("module.", ""): value for key, value in state_dict.items()}
model.load_state_dict(state_dict)

In [ ]:
train_x, train_y = build_tabular_dataset(model, train_dataset, method="lf")
test_x, test_y = build_tabular_dataset(model, test_dataset, method="lf")

train_fusion_set = InputOutputDataset(train_x, train_y)
test_fusion_set = InputOutputDataset(test_x, test_y)

train_fusion_loader = DataLoader(train_fusion_set, batch_size=16)
test_fusion_loader = DataLoader(test_fusion_set, batch_size=16)

## XGBoost

In [ ]:
boost = xgb.XGBClassifier(tree_method='gpu_hist', objective="binary:logistic")

In [ ]:
param_grid = {'max_depth': [3,6,10],
           'learning_rate': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.3],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

clf = GridSearchCV(estimator=boost, 
                   param_grid=param_grid,
                   scoring='roc_auc', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

In [ ]:
with open(MODELS_PATH+'/LateFusionXGBParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

## SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC()

In [ ]:
param_grid = [
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
    {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'degree': [1,2,3], 'kernel': ['poly']}
]

clf = GridSearchCV(estimator=svm, 
                   param_grid=param_grid,
                   scoring='roc_auc', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

In [ ]:
with open(MODELS_PATH+'/LateFusionSVMParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

## FCNN

In [ ]:
fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=8, out_features=32),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()
model = HyMNet(tabular_model=fusion_model)

In [ ]:
optimizer = torch.optim.AdamW([
    {'params': fusion_model.parameters(), 'lr': 0.005}],
)

In [ ]:
epochs = 250
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epoch_length * epochs, eta_min=0)
metrics, best_model = train_val(epochs=epochs, model=model, criterion=criterion, optimizer=optimizer, train_loader=train_fusion_loader,
                    val_loader=test_fusion_loader, scheduler=scheduler, device=device, save_model=True)

In [ ]:
torch.save(best_model, MODELS_PATH + "\LateFusionFCNN.pth")

# VotingFusion (Late Fusion)

In [ ]:
# Load image and Tabular model
image_model = get_retfound("/home/baharoon/HTN/RETFound_cfp_weights.pth", image_size=512,
                          classes=1)

tabular_model = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=8, out_features=32),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=32, out_features=16),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=16, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=8, out_features=1),
)


# load fusion model
tabular_model_fusion = nn.Sequential(
    nn.Linear(in_features=2, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(p=0.5),
    nn.Linear(in_features=8, out_features=32),
)

image_model_fusion = get_retfound("/home/baharoon/HTN/RETFound_cfp_weights.pth", image_size=512,
                          classes=8).requires_grad_(True)

fusion_model_fusion = nn.Sequential(
    nn.Linear(in_features=40, out_features=128),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=128, out_features=32),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=1),
)

fusion = HyMNet(image_model=image_model_fusion, tabular_model=tabular_model_fusion, fusion_model=fusion_model_fusion)
state_dict = torch.load(MODELS_PATH + "/JointFusion_finetune.pth")
state_dict = {key.replace("module.", ""): value for key, value in state_dict.items()}
fusion.load_state_dict(state_dict)

model = HyMNet(image_model=image_model, tabular_model=tabular_model, fusion_model=fusion).cuda()

state_dict = torch.load(FM_PATH)
state_dict = {key.replace("module.", ""): value for key, value in state_dict.items()}
model.load_state_dict(state_dict, strict=False)

state_dict = torch.load(DM_PATH)
state_dict = {key.replace("module.", ""): value for key, value in state_dict.items()}
model.load_state_dict(state_dict, strict=False)

In [ ]:
train_x, train_y = build_tabular_dataset(model, train_dataset, method="vf")
test_x, test_y = build_tabular_dataset(model, test_dataset, method="vf")

train_fusion_set = InputOutputDataset(train_x, train_y)
test_fusion_set = InputOutputDataset(test_x, test_y)

train_fusion_loader = DataLoader(train_fusion_set, batch_size=32)
test_fusion_loader = DataLoader(test_fusion_set, batch_size=32)

## Classifier using all scores

### XGBOOST

In [ ]:
boost = xgb.XGBClassifier(tree_method='gpu_hist', objective="binary:logistic")

In [ ]:
param_grid = {'max_depth': [3,6,10],
           'learning_rate': [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.3],
           'n_estimators': [100, 500, 1000],
           'colsample_bytree': [0.3, 0.7]}

clf = GridSearchCV(estimator=boost, 
                   param_grid=param_grid,
                   scoring='roc_auc', 
                   verbose=1)

clf.fit(train_x, train_y)

print(clf.best_score_)

In [ ]:
with open(MODELS_PATH+'/VotingFusionXGBParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

### SVM

In [ ]:
svm = SVC()

In [ ]:
param_grid = [
    {'C': [0.1, 1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']},
    {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001], 'degree': [1], 'kernel': ['poly']}
]

clf = GridSearchCV(estimator=svm, 
                   param_grid=param_grid,
                   scoring='roc_auc', 
                   verbose=3)

clf.fit(train_x, train_y)

print(clf.best_score_)

In [ ]:
with open(MODELS_PATH+ '/VotingFusionSVMParams.json', 'w', encoding='utf-8') as f:
    json.dump(clf.best_params_, f, ensure_ascii=False, indent=4)

### FCNN

In [ ]:
fusion_model = nn.Sequential(
    nn.Linear(in_features=3, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=8, out_features=32),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=32, out_features=16),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=16, out_features=8),
    nn.LeakyReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(in_features=8, out_features=1),
)

fusion_model = fusion_model.to(device).float()
model = HyMNet(tabular_model=fusion_model)

In [ ]:
optimizer = torch.optim.AdamW([
    {'params': fusion_model.parameters(), 'lr': 0.005}],
)

In [ ]:
epochs = 250
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epoch_length * epochs, eta_min=0)
metrics, best_model = train_val(epochs=epochs, model=model, criterion=criterion, optimizer=optimizer, train_loader=train_fusion_loader,
                    val_loader=test_fusion_loader, scheduler=scheduler, device=device, save_model=True)

In [ ]:
torch.save(best_model, MODELS_PATH + "/VotingFusionFCNN.pth")